# Pancreas endocrinogenesis - scVelo E15.5

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import scvelo as scv
import cellrank as cr
from cellrank.kernels import ConnectivityKernel, VelocityKernel
from cellrank.estimators import GPCCA

## General settings

In [ ]:
# set verbosity levels
sc.settings.verbosity = 2
cr.settings.verbosity = 4
scv.settings.verbosity = 3

In [ ]:
scv.settings.set_figure_params('scvelo', dpi_save=400, dpi=80, transparent=True, fontsize=20, color_map='viridis')

In [ ]:
cr.logging.print_versions()

In [ ]:
PROJECT_DIR = Path("/lustre/groups/ml01/projects/2022_neural_ot/")

## Data loading

In [ ]:
adata = sc.read(PROJECT_DIR / "adata_pancreas_2019.h5ad")
adata = adata[adata.obs['day'] == '15.5', :].copy()

adata.obsp = {}
adata.uns = {}
adata.obsm = {}
adata.varm = {}
adata.obs = adata.obs.loc[:, ['day', 'celltype']]
adata.var = adata.var[[]]

adata.X = adata.layers['spliced'].copy()

adata

## Data pre-processing

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata)

In [ ]:
sc.tl.umap(adata)

In [ ]:
scv.pl.scatter(adata, color='celltype')

## RNA velocity estimation

In [ ]:
scv.tl.recover_dynamics(adata, n_jobs=8)

In [ ]:
scv.tl.velocity(adata, mode='dynamical')

## CellRank

### Transition probabilities

In [ ]:
vk = VelocityKernel(adata).compute_transition_matrix()
ck = ConnectivityKernel(adata).compute_transition_matrix()

combined_kernel = 0.8 * vk + 0.2 * ck

### Macrostate calculation

In [ ]:
g = GPCCA(combined_kernel)

In [ ]:
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)

__<span style="color: red;">TODO: Specify number of states</span>__

In [ ]:
g.compute_macrostates(n_states=4, cluster_key="celltype")

In [ ]:
g.plot_macrostates(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

### Fate probabilities

__<span style="color: red;">TODO: Set terminal states</span>__

In [ ]:
g.set_terminal_states_from_macrostates(
    [
        
    ]
)

In [ ]:
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, time_to_absorption='all')

In [ ]:
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=3)

__<span style="color: red;">TODO: Update lineages and clusters</span>__

In [ ]:
cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=[],
    cluster_key='celltype',
    clusters=[],
    figsize=(10, 4)
)